In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from astropy.table import Table
import astropy.units as u

from lightkurve import search_lightcurve

from gadfly import (
    Hyperparameters, PowerSpectrum,
    StellarOscillatorKernel, ShotNoiseKernel,
    SolarOscillatorKernel
)

In [ ]:
sequence = Table.read('../../docs/gadfly/data/huber2011_sample.ecsv', format='ascii.ecsv')

In [ ]:
# columns to unpack from the table:
cols = ['KIC', 'mass', 'rad', 'teff', 'lum', 'cad_2']

# first get each light curve:
light_curves = []
for i, row in enumerate(sequence[cols]):
    kic, mass, rad, temp, lum, cad = row
    name = f'KIC {kic}'
    cadence_str = 'short' if cad == 0 else 'long'
    light_curve = search_lightcurve(
        name, mission='Kepler',
        cadence=cadence_str
    ).download_all()
    light_curves.append(light_curve)

In [ ]:
# now plot them each:
fig, axes = plt.subplots(1, 4, figsize=(12, 4.25), sharey=True, sharex=True, dpi=350)

solar = SolarOscillatorKernel()
freq = np.logspace(-1, 3, int(1e3)) * u.uHz

for i, row in enumerate(sequence[cols]):
    ax = axes[i]
    kic, mass, rad, temp, lum, cad = row
    name = f'KIC {kic}'
    hp = Hyperparameters.for_star(
        mass * u.M_sun, rad * u.R_sun, 
        temp * u.K, lum * u.L_sun, 
        name=name, quiet=True
    )

    cadence_str = 'short' if cad == 0 else 'long'
    light_curve = search_lightcurve(
        name, mission='Kepler', #quarter=range(6),
        cadence=cadence_str
    ).download_all()

    kernel = (
        StellarOscillatorKernel(hp) +
        # add in a kernel for Kepler shot noise
        ShotNoiseKernel.from_kepler_light_curve(light_curve)
    )

    ps = PowerSpectrum.from_light_curve(
        light_curve, name=name,
        detrend_poly_order=5, 
    ).bin(300)

    kernel_kw = dict(color="k", alpha=0.9, rasterized=True)
    mfc = 'none'
    obs_kw = dict(color=f"C{i}", marker='o', lw=0, ms=4, mfc=mfc, rasterized=True)
    kernel.plot(
        ax=ax,
        p_mode_inset=False,
        freq=freq,
        obs=ps,
        obs_kwargs=obs_kw,
        kernel_kwargs=kernel_kw,
        n_samples=1e4,
        title=name, 
        legend=False
    )
    
    note = (
        f"$M = {mass:.1f}$ M$_\\odot$\n"
        f"$R = {rad:.1f}$ R$_\\odot$\n"
        f"$T_{{\\rm eff}} = {temp:.0f}$ K\n"
        f"$L = {lum:.1f}$ L$_\\odot$"
    )
    
    ax.annotate(note, (2, 0.9), fontsize=9, ha='left', va='bottom')
    if i:
        ax.set_ylabel(None)
    for sp in ['right', 'top']:
        ax.spines[sp].set_visible(False)

ax.set_xlim([1, 1000])
ax.set_ylim([0.5, 1e4])
fig.tight_layout()
fig.savefig('plots/validation-evolution.pdf', bbox_inches='tight')